# Monte Carlo 방법

이 notebook에서는 다양한 Monte Carlo(MC) 알고리즘을 직접 구현해본다.

시작에 필요한 코드를 제공하지만 힌트들 지우고 직접 처음부터 작성해보는 것도 추천한다.

### Part 0: BlackjackEnv 경험해보기

시작은 필요한 package를 import 시킨다.

In [ ]:
import sys
import gym
import numpy as np
from collections import defaultdict

from plot_utils import plot_blackjack_values, plot_policy

아래 코드를 사용하여 [Blackjack](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py) environment의 인스턴스를 생성.

In [ ]:
env = gym.make('Blackjack-v0')

각 state는 다음과 같은 3-tuple 로 구성:
- the player's current sum $\in \{0, 1, \ldots, 31\}$,
- the dealer's face up card $\in \{1, \ldots, 10\}$, and
- whether or not the player has a usable ace (`no` $=0$, `yes` $=1$).

agent는 2가지 가능한 actions을 가진다:

```
    STICK = 0
    HIT = 1
```
아래 코드를 실행하여 확인

In [ ]:
print(env.observation_space)
print(env.action_space)

아래 cell 코드를 실행하면 random polity로 Blackjack을 play한다.

(_아래 코드는 Blackjack을 3번 play한다. - 이 숫자를 바꾸거나 아래 cell을 여러번 실행할 수 있다.  해당  cell은 agent가 environment와 상호작용하여 반환되는 결과를 경험할 수 있도록 구현하였다._)

In [ ]:
for i_episode in range(3):
    state = env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break

### Part 1: MC Prediction

이 섹션에서 MC prediction(action-value function을 estimation하기) 구현부를 직접 작성해보자.

player는 카드의 합이 18을 넘으면 _거의_ 항상 stick되는 policy를 보게된다. 특히 sum이 18을 넘으면 `STICK` action을 80% 확률로 선택한다. sum이 18 혹은 아래면 80% 확률로 `HIT` action을 선택한다. `generate_episode_from_limit_stochastic` 함수는 이 policy를 사용하여 episode를 샘플한다.

이 함수는 **input**으로 :
- `bj_env`: OpenAI Gym의 Blackjack environment의 인스턴스이다.

**output**으로 반환 :
- `episode`: This is a list of (state, action, reward) tuples (of tuples) and corresponds to $(S_0, A_0, R_1, \ldots, S_{T-1}, A_{T-1}, R_{T})$, where $T$ is the final time step.  In particular, `episode[i]` returns $(S_i, A_i, R_{i+1})$, and `episode[i][0]`, `episode[i][1]`, and `episode[i][2]` return $S_i$, $A_i$, and $R_{i+1}$, respectively.

In [ ]:
def generate_episode_from_limit_stochastic(bj_env):
    episode = []
    state = bj_env.reset()
    while True:
        probs = [0.8, 0.2] if state[0] > 18 else [0.2, 0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

해당 policy로 Blackjack을 play하기 위해서 아래 코드 cell을 실행.

(*이 코드는 Blackjack을 3번 play한다. - 이 숫자를 바꾸거나 cell을 여러번 실행한다.  `generate_episode_from_limit_stochastic` 함수의 출력에 대해서 친숙해지라고 이 cell을 제공한다.*)

In [ ]:
for i in range(3):
    print(generate_episode_from_limit_stochastic(env))

이제 MC prediciton의 구현부를 직접 작성할 준비가 되었다. first-visit이나 every-visit MC prediction 중에 하나를 자유롭게 구현해보자. Blackjack environment의 경우도 동일한 기술은 동일하다.

알고리즘은 다음의 3개 인자를 갖는다:
- `env`: OpenAI Gym environment의 instance.
- `num_episodes`: episode의 개수로 agent-environment 상호작용을 통해서 생성.
- `generate_episode`: 상호작용 episode의 반환하는 함수
- `gamma`: discount rate. 0과 1사이의 값을 갖는다. (default value: `1`).

알고리즘이 출력으로 반환 :
- `Q`: dictionary(1차원 배열)로 `Q[s][a]`는 state `s`와 action `a`에 대한 estimated action value이다.

In [ ]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=1.0):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        ## TODO: complete the function
        
    return Q

아래 cell을 이용해서 $Q$를 estimate하는 action-value function을 얻는다. state-value 함수를 그래프가 나온다.

구현의 정확도를 검증하기 위해서 아래 그래프와 **Monte_Carlo_Solution.ipynb** 의 그래프를 비교해보자.

In [ ]:
# obtain the action-value function
Q = mc_prediction_q(env, 500000, generate_episode_from_limit_stochastic)

# obtain the corresponding state-value function
V_to_plot = dict((k,(k[0]>18)*(np.dot([0.8, 0.2],v)) + (k[0]<=18)*(np.dot([0.2, 0.8],v))) \
         for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V_to_plot)

### Part 2: MC Control

이 섹션에서 constant-$\alpha$ MC control 를 직접 구현해보자.

알고리즘의 4개 인자:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: step을 업데이트하기 위한 step-size parameter
- `gamma`: discount rate. 0과 1사이 값으로 (default value: `1`).

알고리즘이 출력으로 반환 :
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: dictionary로 `policy[s]`는 state `s`를 observing한 후에 agent가 선택하는 action을 반환한다.

(_원하면 추가 함수를 정의하여 작성 가능_)

In [ ]:
def mc_control(env, num_episodes, alpha, gamma=1.0):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(nA))
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        ## TODO: complete the function
        
    return policy, Q

아래 cell을 사용하여 estimated optimal policy와 action-value function를 얻을 수 있다.  `num_episodes` 와 `alpha` parameters에는 직접 값을 넣어야만 하는 것을 명심하자!

In [ ]:
# obtain the estimated optimal policy and action-value function
policy, Q = mc_control(env, ?, ?)

다음으로 corresponding state-value function 그래프를 그린다.

In [ ]:
# obtain the corresponding state-value function
V = dict((k,np.max(v)) for k, v in Q.items())

# plot the state-value function
plot_blackjack_values(V)

마지막으로 최적으로 estimation된 policy를 시각화한다.

In [ ]:
# plot the policy
plot_policy(policy)

**true** optimal policy $\pi_*$는  [textbook](http://go.udacity.com/rl-textbook)의 Figure 5.2에서 볼 수 있다. 최종 optimal policy와 비교해보면 결과가 만족스러운지 확인하자. 알고리즘 성능에 만족하지 않는다면 decay rate of $\epsilon$, $\alpha$ 값을 변경하거나 더 많은 episode들에 대해서 알고리즘을 실행하여 더 낮은 결과를 얻을 수 있다.

![True Optimal Policy](images/optimal.png)